In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os


In [2]:
filenameAML = 'RawAML/aml_log_2024-06-11_07-30-58.aml'

In [3]:
header_data = []
sensor_meta_data = []
measurement_metadata = []
measurement_data = []
current_section = ''

with open(filenameAML, 'r') as file:
    lines = file.readlines()

for line in lines:
    line = line.strip()
    if line.startswith('[Header]'):
        current_section = 'Header'
        continue
    elif line.startswith('[SensorMetaData]'):
        current_section = 'SensorMetaData'
        continue
    elif line.startswith('[MeasurementMetadata]'):
        current_section = 'MeasurementMetadata'
        continue
    elif line.startswith('[MeasurementData]'):
        current_section = 'MeasurementData'
        continue
    
    if current_section == 'Header':
        header_data.append(line)
    elif current_section == 'SensorMetaData':
        sensor_meta_data.append(line)
    elif current_section == 'MeasurementMetadata':
        measurement_metadata.append(line)
    elif current_section == 'MeasurementData':
        measurement_data.append(line)

In [4]:
measurement_data = [data for data in measurement_data if data]

measurement_metadata_header = measurement_metadata[0].split(',')
measurement_units = measurement_metadata[1].split(',')

measurement_array = [line.split(',') for line in measurement_data]
aml_df = pd.DataFrame(measurement_array, columns=['date', 'time'] + measurement_metadata_header[2:])
#measurement_df['datetime'] = pd.to_datetime(measurement_df['date'] + ' ' + measurement_df['time'])

aml_df

,date,time,Vbat,Cond,TempCT,Pressure,DO,TempDO,DOM,Chloro-blue,Turbidity,analog,Salinity,Density,CalcSV,Depth
0,2024-06-11,07:30:58.77,8.19,19.003,15.014,-0.10961,287.21,15.34,1.95,0.78,8.97,0.00,14.258452,1010.041199,1482.677124,-0.10861
1,2024-06-11,07:30:58.82,8.19,20.107,14.934,-0.13114,287.21,15.34,2.42,1.21,8.84,0.00,15.194602,1010.772461,1483.493408,-0.12995
2,2024-06-11,07:30:58.92,8.19,20.493,14.837,-0.08658,287.21,15.34,3.73,1.54,5.34,0.00,15.551564,1011.063660,1483.581787,-0.08579
3,2024-06-11,07:30:58.97,8.19,20.770,14.786,-0.05829,287.21,15.34,4.09,1.58,5.88,0.00,15.802451,1011.265381,1483.699707,-0.05776
4,2024-06-11,07:30:59.12,8.19,20.749,14.751,-0.01125,287.17,15.34,5.18,1.76,3.67,0.00,15.798581,1011.268921,1483.578857,-0.01115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211,2024-06-11,07:32:50.52,8.19,7.863,14.743,-0.06167,290.86,14.73,5.17,1.77,68.63,0.00,5.530354,1003.399292,1471.600342,-0.06111
1212,2024-06-11,07:32:50.72,8.19,0.195,14.700,-0.03796,290.86,14.73,3.10,42.52,62.91,0.00,0.115823,999.235535,1465.030151,-0.03762
1213,2024-06-11,07:32:50.82,8.19,0.026,14.689,-0.07525,290.84,14.72,2.23,55.09,34.73,0.00,0.014192,999.158020,1464.867432,-0.07456
1214,2024-06-11,07:32:50.87,8.19,0.000,14.686,-0.05432,290.84,14.72,1.74,58.39,27.38,0.00,-99.989998,-99.989998,-99.989998,-0.05382


In [5]:
aml_df.iloc[:, 2:] = aml_df.iloc[:, 2:].apply(lambda x: x.astype(float))

In [6]:
max_depth = aml_df['Depth'].max()
idx_max = aml_df['Depth'].idxmax()

# Filter out the upcast phase data
aml_df = aml_df.loc[:aml_df['Depth'].idxmax()]
aml_df


,date,time,Vbat,Cond,TempCT,Pressure,DO,TempDO,DOM,Chloro-blue,Turbidity,analog,Salinity,Density,CalcSV,Depth
0,2024-06-11,07:30:58.77,8.19,19.003,15.014,-0.10961,287.21,15.34,1.95,0.78,8.97,0.0,14.258452,1010.041199,1482.677124,-0.10861
1,2024-06-11,07:30:58.82,8.19,20.107,14.934,-0.13114,287.21,15.34,2.42,1.21,8.84,0.0,15.194602,1010.772461,1483.493408,-0.12995
2,2024-06-11,07:30:58.92,8.19,20.493,14.837,-0.08658,287.21,15.34,3.73,1.54,5.34,0.0,15.551564,1011.06366,1483.581787,-0.08579
3,2024-06-11,07:30:58.97,8.19,20.77,14.786,-0.05829,287.21,15.34,4.09,1.58,5.88,0.0,15.802451,1011.265381,1483.699707,-0.05776
4,2024-06-11,07:30:59.12,8.19,20.749,14.751,-0.01125,287.17,15.34,5.18,1.76,3.67,0.0,15.798581,1011.268921,1483.578857,-0.01115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,2024-06-11,07:31:42.57,8.19,33.386,6.598,20.21006,234.36,6.65,4.01,6.32,2.12,0.0,33.292515,1026.220459,1475.279175,20.02554
543,2024-06-11,07:31:42.72,8.19,33.389,6.599,20.23948,234.36,6.65,3.98,5.65,2.16,0.0,33.295315,1026.222534,1475.28833,20.05469
544,2024-06-11,07:31:42.77,8.19,33.389,6.598,20.26245,234.36,6.65,4.02,4.35,2.17,0.0,33.296513,1026.223755,1475.2854,20.07745
545,2024-06-11,07:31:42.87,8.19,33.387,6.598,20.28248,234.36,6.65,3.99,3.53,2.27,0.0,33.294281,1026.222168,1475.281982,20.0973


In [8]:
# plt.figure(figsize=(10, 6))
# plt.plot(downcast_data['TempCT'], downcast_data['Depth'].astype(float), marker='o', linestyle='-', color='b')
# plt.xlabel('Temperature')
# plt.ylabel('Depth (m)')
# plt.title('Depth vs. Datetime (Downcast Phase)')
# plt.grid(True)
# plt.ylim(min(downcast_data['Depth'].astype(float)), max(downcast_data['Depth'].astype(float)))
# plt.gca().invert_yaxis()
# plt.tight_layout()
# plt.show()

In [12]:
samples_df = pd.read_excel('DataCollection_m.xlsx',sheet_name='Bottles depth-all stations')
samples_df = samples_df.drop(['Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17'],axis=1)
samples_df['Timestamp'] = pd.to_datetime(samples_df['Timestamp'])

samples_df.columns

Index(['Sample', 'Station', 'Date', 'Time', 'Timestamp', 'Depth',
       'number of bottles fired', 'Seafloor depth', 'Latitude_SEAB', 'T',
       'New position', 'Latitude', 'Longitude'],
      dtype='object')

In [100]:
folder_path = 'RawAML/'

In [101]:
dfs = []
i=0

# Loop through each file in the directory
for filename in os.listdir(folder_path):
    if filename.endswith(".aml"):
        filepath = os.path.join(folder_path, filename)
        #print(filename)
        # Initialize variables for each file
        header_data = []
        sensor_meta_data = []
        measurement_metadata = []
        measurement_data = []
        current_section = ''
        
        with open(filepath, 'r') as file:
            lines = file.readlines()
        
        for line in lines:
            line = line.strip()
            if line.startswith('[Header]'):
                current_section = 'Header'
                continue
            elif line.startswith('[SensorMetaData]'):
                current_section = 'SensorMetaData'
                continue
            elif line.startswith('[MeasurementMetadata]'):
                current_section = 'MeasurementMetadata'
                continue
            elif line.startswith('[MeasurementData]'):
                current_section = 'MeasurementData'
                continue
            
            if current_section == 'Header':
                header_data.append(line)
            elif current_section == 'SensorMetaData':
                sensor_meta_data.append(line)
            elif current_section == 'MeasurementMetadata':
                measurement_metadata.append(line)
            elif current_section == 'MeasurementData':
                measurement_data.append(line)
        
        measurement_data = [data for data in measurement_data if data]
        
        measurement_metadata_header = measurement_metadata[0].split(',')
        measurement_units = measurement_metadata[1].split(',')
        
        measurement_array = [line.split(',') for line in measurement_data]
        measurement_df = pd.DataFrame(measurement_array, columns=['date', 'time'] + measurement_metadata_header[2:])
        
        # Convert relevant columns to float
        measurement_df.iloc[:, 2:] = measurement_df.iloc[:, 2:].apply(lambda x: pd.to_numeric(x, errors='coerce'))
        
        # Extract GPS coordinates from header or sensor meta data
        lat = None
        lon = None
        for line in header_data:
            if line.startswith('GPSLatitude'):
                lat = line.split('=')[1].strip()
            elif line.startswith('GPSLongitude'):
                lon = line.split('=')[1].strip()

        if lat is not None and lon is not None:
            if lat == 'NL':
                measurement_df['Latitude'] = None
                measurement_df['Longitude'] = None
            else:
                measurement_df['Latitude'] = float(lat)
                measurement_df['Longitude'] = float(lon)
        
        # Keep only downcast
        max_depth = measurement_df['Depth'].max()
        measurement_df = measurement_df.loc[:measurement_df['Depth'].idxmax()]
        measurement_df['Timestamp'] = pd.to_datetime(measurement_df['date'] + ' ' + measurement_df['time'])
        measurement_df = measurement_df.drop(['date', 'time'], axis=1)

        # Find respective station number and add station number
        measurement_timestamp = measurement_df['Timestamp'].iloc[0]
        closest_sample_row = None

        for idx, row in samples_df.iterrows():
            if (abs(measurement_timestamp - row['Timestamp']).total_seconds()) < 300:
                closest_sample_row = row
                print(abs((measurement_timestamp -row['Timestamp']).total_seconds()))
                print('Found closest sample row:', closest_sample_row['Station'])
                break

        if closest_sample_row is not None:
            print('Storing...')
            station = closest_sample_row['Station']
            measurement_df['Station'] = station
            print('Stored')
        else:
            print('No matching timestamp found in samples_df within the tolerance.')


        dfs.append(measurement_df)
        i=i+2


58.77
Found closest sample row: 308
Storing...
Stored
34.61
Found closest sample row: 309
Storing...
Stored
19.43
Found closest sample row: 310
Storing...
Stored
10.7
Found closest sample row: 311
Storing...
Stored
2.13
Found closest sample row: 312
Storing...
Stored
33.21
Found closest sample row: 313
Storing...
Stored
280.16
Found closest sample row: 313
Storing...
Stored
17.94
Found closest sample row: 316
Storing...
Stored
25.5
Found closest sample row: 318
Storing...
Stored
36.07
Found closest sample row: 319
Storing...
Stored
13.17
Found closest sample row: 320
Storing...
Stored
14.92
Found closest sample row: 320
Storing...
Stored
10.93
Found closest sample row: 321
Storing...
Stored
142.48
Found closest sample row: 321
Storing...
Stored
36.3
Found closest sample row: 322
Storing...
Stored
26.23
Found closest sample row: 323
Storing...
Stored
No matching timestamp found in samples_df within the tolerance.
No matching timestamp found in samples_df within the tolerance.
36.65
Foun

In [102]:
aml_df=0
aml_df = pd.concat(dfs, ignore_index=True)
aml_df = aml_df[['Station', 'Timestamp', 'Latitude', 'Longitude', 'Vbat', 'Cond', 'TempCT', 'Pressure', 'DO', 'TempDO', 'DOM',
       'Chloro-blue', 'Turbidity', 'analog', 'Salinity', 'Density', 'CalcSV',
       'Depth']]
aml_df = aml_df[(aml_df['Depth'] >= 0)]

C:\Users\franc\AppData\Local\Temp\ipykernel_32172\893492938.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aml_df = pd.concat(dfs, ignore_index=True)


In [103]:
filenameRBR = 'Raw/231420_20240614_1557.csv'

In [104]:
rbr_df = pd.read_csv(filenameRBR, skiprows=7, encoding='latin1')

rbr_df = rbr_df.rename(columns={'// timestamp(yyyy-mm-ddTHH:MM:ss.FFF)': 'Timestamp'})
rbr_df['Timestamp'] = pd.to_datetime(rbr_df['Timestamp'])
rbr_df = rbr_df[(rbr_df['    depth(m)'] >= 0)]


In [105]:
time_diff = rbr_df['Timestamp'].diff()
timestamps = rbr_df.loc[(time_diff > pd.Timedelta(seconds=10)) | (time_diff.isna()), 'Timestamp']

groups = []
for i in range(len(timestamps)):
    if i == len(timestamps) - 1:
        # Last timestamp case
        group = rbr_df[rbr_df['Timestamp'] >= timestamps.iloc[i]]
    else:
        # Regular case
        group = rbr_df[(rbr_df['Timestamp'] >= timestamps.iloc[i]) & (rbr_df['Timestamp'] < timestamps.iloc[i + 1])]
    
    # Find max depth value within the group
    max_depth = group['    depth(m)'].max()
    
    # Discard rows after max depth within the group
    max_depth_index = group[group['    depth(m)'] == max_depth].index[0]  # Assuming depth values are unique within each group
    group = group.loc[group.index <= max_depth_index]
    
    # Append filtered group to groups list
    groups.append(group)

# Concatenate remaining groups into new rbr_df
rbr_df = pd.concat(groups, ignore_index=True)

In [106]:
# Add coordinates to RBR as well
rbr_df['Latitude'] = np.nan
rbr_df['Longitude'] = np.nan
rbr_df['Station'] = ''

max_time_diff_minutes = 10

# Loop through each row in RBR and find matching timestamp in AML
for rbr_index, rbr_row in rbr_df.iterrows():
    rbr_timestamp = rbr_row['Timestamp']
    
    # Find matching timestamp in AML within max_time_diff_minutes
    matching_row = aml_df[(aml_df['Timestamp'] - rbr_timestamp).abs() <= pd.Timedelta(minutes=max_time_diff_minutes)]
    
    if not matching_row.empty:
        # Assign coordinates from AML to RBR
        rbr_df.at[rbr_index, 'Latitude'] = matching_row['Latitude'].values[0]
        rbr_df.at[rbr_index, 'Longitude'] = matching_row['Longitude'].values[0]
        rbr_df.at[rbr_index, 'Station'] = matching_row['Station'].values[0]


In [107]:
rbr_df = rbr_df[['Timestamp','Station', 'Latitude', 'Longitude', '    depth(m)', '    conductivity(mS/cm)', '    temperature(°C)',
       '    pressure(dbar)', '    temperature1(°C)',
       '    dissolved_o2_concentration(µmol/L)', '    chlorophyll-a(ug/l)',
       '    fdom(ppb)', '    turbidity(FTU)', '    voltage(V)',
       '    voltage1(V)', '    voltage2(V)', '    voltage3(V)',
       '    sea_pressure(dbar)', '    salinity(PSU)',
       '    speed_of_sound(m/s)', '    specific_conductivity(µS/cm)',
       '    dissolved_o2_saturation(%)']]


In [108]:
rbr_df.columns

Index(['Timestamp', 'Station', 'Latitude', 'Longitude', '    depth(m)',
       '    conductivity(mS/cm)', '    temperature(°C)', '    pressure(dbar)',
       '    temperature1(°C)', '    dissolved_o2_concentration(µmol/L)',
       '    chlorophyll-a(ug/l)', '    fdom(ppb)', '    turbidity(FTU)',
       '    voltage(V)', '    voltage1(V)', '    voltage2(V)',
       '    voltage3(V)', '    sea_pressure(dbar)', '    salinity(PSU)',
       '    speed_of_sound(m/s)', '    specific_conductivity(µS/cm)',
       '    dissolved_o2_saturation(%)'],
      dtype='object')

In [109]:
aml_df.columns

Index(['Station', 'Timestamp', 'Latitude', 'Longitude', 'Vbat', 'Cond',
       'TempCT', 'Pressure', 'DO', 'TempDO', 'DOM', 'Chloro-blue', 'Turbidity',
       'analog', 'Salinity', 'Density', 'CalcSV', 'Depth'],
      dtype='object')

In [110]:
samples_df = pd.read_excel('DataCollection_m.xlsx',sheet_name='Bottles depth-all stations')
samples_df = samples_df.drop(['Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16'],axis=1)
samples_df['Timestamp'] = pd.to_datetime(samples_df['Timestamp'])

samples_df.drop(['number of bottles fired','New position'],axis=1)
samples_df['Temperature'] = np.nan
samples_df['Salinity'] = np.nan
samples_df['Fluorescence'] = np.nan
samples_df['NPOC'] = np.nan
samples_df['TN'] = np.nan
samples_df = samples_df[['Sample', 'Station', 'Timestamp', 'Latitude', 
                         'Longitude', 'Depth', 'Seafloor depth','Salinity',
                         'Temperature','Fluorescence','NPOC','TN']]

numeric_columns = samples_df.columns.drop(['Timestamp','Sample'])  # Exclude 'Timestamp' column
samples_df[numeric_columns] = samples_df[numeric_columns].apply(pd.to_numeric, errors='coerce')


samples_df.columns

Index(['Sample', 'Station', 'Timestamp', 'Latitude', 'Longitude', 'Depth',
       'Seafloor depth', 'Salinity', 'Temperature', 'Fluorescence', 'NPOC',
       'TN'],
      dtype='object')

In [111]:
toc_df = pd.read_excel('Raw/NPOC_corrected.ods')
tn_df = pd.read_excel('Raw/TN_corrected.ods')

In [112]:
fdom_df = pd.read_excel('Raw/oresund_FDOM.xlsx')

numeric_columns = fdom_df.columns.drop('sample') 
fdom_df[numeric_columns] = fdom_df[numeric_columns].apply(pd.to_numeric, errors='coerce')


fdom_df = fdom_df[['sample', 'T', 'A', 'M', 'C', 'D', 'N']]

sample_counts = fdom_df['sample'].str.split().str[0].value_counts()
mask = fdom_df['sample'].apply(lambda x:  (x.split()[0].endswith('S') or x.split()[0].endswith('B')) 
                               and ((sample_counts[x.split()[0]] > 1 and x.split()[1] == '(02)') 
                               or (sample_counts[x.split()[0]] == 1)) )
fdom_df = fdom_df[mask]

# fdom_df


In [113]:

for s in samples_df['Station'].unique():

    aml_depth = aml_df.loc[aml_df['Station'] == s, 'Depth'].max()
    rbr_depth = rbr_df.loc[rbr_df['Station'] == s, '    depth(m)'].max()
    max_depth = max(aml_depth, rbr_depth)

    if samples_df.loc[samples_df['Station'] == s, 'Seafloor depth'].isnull().any():
        print(f"Updating Seafloor depth for Station {s}")

        samples_df.loc[samples_df['Station'] == s, 'Seafloor depth'] = max_depth

    if pd.isnull(samples_df.loc[samples_df['Station'] == s, 'Depth']).any():
        print(f"Updating Depth of Measurement for Station {s}")

        for idx, row in samples_df[samples_df['Station'] == s].iterrows():
            if row['Sample'].endswith('S') or row['Sample'].endswith('SF'):
                samples_df.loc[idx, 'Depth'] = 3
            elif row['Sample'].endswith('B') or row['Sample'].endswith('BF'):
                samples_df.loc[idx, 'Depth'] = max_depth - 1.5


    if pd.isnull(samples_df.loc[samples_df['Station'] == s, 'Temperature']).any():
        print(f"Updating Temperature of Measurement for Station {s}")

        for idx, row in samples_df[samples_df['Station'] == s].iterrows():
            sample_depth = row['Depth']
            total_temp = 0
            count = 0

            # Search aml_df for temperatures around sample_depth
            aml_temps = aml_df[(aml_df['Station'] == s) & 
                               (aml_df['Depth'] >= sample_depth - 1) & 
                               (aml_df['Depth'] <= sample_depth + 1)]['TempCT']
            if not aml_temps.empty:
                total_temp += aml_temps.mean()
                count += 1
            
            else:
                sample_depth = max_depth
                aml_temps = aml_df[(aml_df['Station'] == s) & 
                               (aml_df['Depth'] >= sample_depth - 1) & 
                               (aml_df['Depth'] <= sample_depth + 1)]['TempCT']
                if not aml_temps.empty:
                    total_temp += aml_temps.mean()
                    count += 1

            # Search rbr_df for temperatures around sample_depth
            rbr_temps = rbr_df[(rbr_df['Station'] == s) & 
                               (rbr_df['    depth(m)'] >= sample_depth - 1) & 
                               (rbr_df['    depth(m)'] <= sample_depth + 1)]['    temperature(°C)']
            if not rbr_temps.empty:
                total_temp += rbr_temps.mean()
                count += 1
            
            else: 
                sample_depth = max_depth
                rbr_temps = rbr_df[(rbr_df['Station'] == s) & 
                               (rbr_df['    depth(m)'] >= sample_depth - 1) & 
                               (rbr_df['    depth(m)'] <= sample_depth + 1)]['    temperature(°C)']
                if not rbr_temps.empty:
                    total_temp += rbr_temps.mean()
                    count += 1
            
            # Calculate overall average temperature if temperatures were found
            if count > 0:
                avg_temp = total_temp / count
            else:
                avg_temp = None  # Handle case where no temperatures were found

            # Update samples_df with avg_temp for the current row
            samples_df.loc[idx, 'Temperature'] = avg_temp
            


    if pd.isnull(samples_df.loc[samples_df['Station'] == s, 'Salinity']).any():
        print(f"Updating Salinity of Measurement for Station {s}")

        for idx, row in samples_df[samples_df['Station'] == s].iterrows():
            sample_depth = row['Depth']
            total_psal = 0
            count = 0

            # Search aml_df for salinities around sample_depth
            aml_psal = aml_df[(aml_df['Station'] == s) & 
                               (aml_df['Depth'] >= sample_depth - 1) & 
                               (aml_df['Depth'] <= sample_depth + 1)]['Salinity']
            if not aml_psal.empty:
                total_psal += aml_psal.mean()
                count += 1
            
            else:
                sample_depth = max_depth
                aml_psal = aml_df[(aml_df['Station'] == s) & 
                               (aml_df['Depth'] >= sample_depth - 1) & 
                               (aml_df['Depth'] <= sample_depth + 1)]['Salinity']
                if not aml_psal.empty:
                    total_psal += aml_psal.mean()
                    count += 1

            # Search rbr_df for salinities around sample_depth
            rbr_psal = rbr_df[(rbr_df['Station'] == s) & 
                               (rbr_df['    depth(m)'] >= sample_depth - 1) & 
                               (rbr_df['    depth(m)'] <= sample_depth + 1)]['    salinity(PSU)']
            if not rbr_psal.empty:
                total_psal += rbr_psal.mean()
                count += 1
            
            else: 
                sample_depth = max_depth
                rbr_psal = rbr_df[(rbr_df['Station'] == s) & 
                               (rbr_df['    depth(m)'] >= sample_depth - 1) & 
                               (rbr_df['    depth(m)'] <= sample_depth + 1)]['    temperature(°C)']
                if not rbr_psal.empty:
                    total_psal += rbr_psal.mean()
                    count += 1
            
            # Calculate overall average temperature if temperatures were found
            if count > 0:
                avg_psal = total_psal / count
            else:
                avg_psal = None  # Handle case where no temperatures were found

            # Update samples_df with avg_temp for the current row
            samples_df.loc[idx, 'Salinity'] = avg_psal
       
            
samples_df['NPOC'] = samples_df.apply(lambda row: toc_df.loc[toc_df['Sample'] == row['Sample'], 'TOC (µM)'].values[0]
                                      if not toc_df.loc[toc_df['Sample'] == row['Sample']].empty
                                      else None,axis=1)
samples_df['TN'] = samples_df.apply(lambda row: tn_df.loc[tn_df['Sample'] == row['Sample'], 'TN (µM)'].values[0]
                                      if not tn_df.loc[tn_df['Sample'] == row['Sample']].empty
                                      else None,axis=1)

# samples_df

Updating Temperature of Measurement for Station 308
Updating Salinity of Measurement for Station 308
Updating Temperature of Measurement for Station 309
Updating Salinity of Measurement for Station 309
Updating Temperature of Measurement for Station 310
Updating Salinity of Measurement for Station 310
Updating Temperature of Measurement for Station 311
Updating Salinity of Measurement for Station 311
Updating Temperature of Measurement for Station 312
Updating Salinity of Measurement for Station 312
Updating Temperature of Measurement for Station 313
Updating Salinity of Measurement for Station 313
Updating Temperature of Measurement for Station 316
Updating Salinity of Measurement for Station 316
Updating Temperature of Measurement for Station 318
Updating Salinity of Measurement for Station 318
Updating Temperature of Measurement for Station 319
Updating Salinity of Measurement for Station 319
Updating Temperature of Measurement for Station 320
Updating Salinity of Measurement for St

In [114]:
samples_df

,Sample,Station,Timestamp,Latitude,Longitude,Depth,Seafloor depth,Salinity,Temperature,Fluorescence,NPOC,TN
0,1B,308,2024-06-11 07:30:00,55.78386,12.74563,19.060000,20.190000,33.302367,6.602990,NaN,139.8,16.1
1,1S,308,2024-06-12 07:30:00,55.78386,12.74563,3.820000,13.810000,16.867784,14.892072,NaN,301.7,15.7
2,2B,309,2024-06-11 08:43:00,55.80377,12.72384,18.870000,28.860000,33.443722,6.520970,NaN,145.2,15.0
3,2S,309,2024-06-11 08:43:00,55.80377,12.72384,4.760000,14.760000,18.838659,14.249816,NaN,297.8,14.0
4,3B,310,2024-06-11 09:09:00,55.82440,12.70632,21.380000,21.880000,33.524142,6.431700,NaN,95.1,16.2
5,3S,310,2024-06-11 09:09:00,55.82440,12.70632,3.690000,13.680000,17.936224,14.583480,NaN,289.7,14.6
6,4B,311,2024-06-11 09:37:00,55.85134,12.68696,13.440000,14.050000,32.939223,6.797599,NaN,128.7,16.6
7,4S,311,2024-06-11 09:37:00,55.85134,12.68696,3.680000,13.680000,19.774416,13.812609,NaN,302.7,14.3
8,5B,312,2024-06-11 09:56:00,55.86998,12.68613,12.520000,13.560000,31.498887,7.591425,NaN,295.5,15.8
9,5S,312,2024-06-11 09:56:00,55.86998,12.68613,4.010000,13.780000,19.302852,13.925634,NaN,262.4,15.6


In [115]:
print(aml_df.columns)
print(rbr_df.columns)
common_columns = set(aml_df.columns).intersection(set(rbr_df.columns))
common_columns

Index(['Station', 'Timestamp', 'Latitude', 'Longitude', 'Vbat', 'Cond',
       'TempCT', 'Pressure', 'DO', 'TempDO', 'DOM', 'Chloro-blue', 'Turbidity',
       'analog', 'Salinity', 'Density', 'CalcSV', 'Depth'],
      dtype='object')
Index(['Timestamp', 'Station', 'Latitude', 'Longitude', '    depth(m)',
       '    conductivity(mS/cm)', '    temperature(°C)', '    pressure(dbar)',
       '    temperature1(°C)', '    dissolved_o2_concentration(µmol/L)',
       '    chlorophyll-a(ug/l)', '    fdom(ppb)', '    turbidity(FTU)',
       '    voltage(V)', '    voltage1(V)', '    voltage2(V)',
       '    voltage3(V)', '    sea_pressure(dbar)', '    salinity(PSU)',
       '    speed_of_sound(m/s)', '    specific_conductivity(µS/cm)',
       '    dissolved_o2_saturation(%)'],
      dtype='object')


{'Latitude', 'Longitude', 'Station', 'Timestamp'}

In [119]:
aml_df['DOM'] = (aml_df['DOM'] - 0.12) * 0.0141
rbr_df['    fdom(ppb)'] = (rbr_df['    fdom(ppb)'] - 1.2) * 0.0114

In [120]:
with pd.ExcelWriter('ctd_data.xlsx', engine='openpyxl') as writer:
    aml_df.to_excel(writer, sheet_name='AML Data', index=False)
    rbr_df.to_excel(writer, sheet_name='RBR Data', index=False)

with pd.ExcelWriter('DataCollection_m.xlsx', engine='openpyxl', mode='a') as writer:
    if 'Samples Data' in writer.book.sheetnames:
        writer.book.remove(writer.book['Samples Data'])
        samples_df.to_excel(writer, sheet_name='Samples Data', index=False)

    if 'Fluorescence' in writer.book.sheetnames:
        writer.book.remove(writer.book['Fluorescence'])
        samples_df.to_excel(writer, sheet_name='Fluorescence', index=False)
    
    # if 'AML filtered' in writer.book.sheetnames:
    #     writer.book.remove(writer.book['AML filtered'])
    #     samples_df.to_excel(writer, sheet_name='AML filtered', index=False)
    
    # if 'RBR filtered' in writer.book.sheetnames:
    #     writer.book.remove(writer.book['RBR filtered'])
    #     samples_df.to_excel(writer, sheet_name='RBR filtered', index=False)
